In [1]:
import arcpy
from arcgis.gis import GIS
import pandas as pd
import numpy as np
from arcgis.features import SpatialDataFrame
from arcpy.sa import *
from arcgis.geocoding import geocode
import pylab as pl
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
g = GIS(url='https://nkwjones.maps.arcgis.com',username='nj935_',password='')

In [3]:
g.map("USA")

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

In [9]:
ESRI_pop = g.content.get('625e9da1afed40b78aaf412f519b22d3')
pop_lyr = ESRI_pop.layers[0]

In [12]:
ESRI_elevation = g.content.get('58a541efc59545e6b7137f961d7de883')
elev_lyr = ESRI_elevation.layers[0]

In [13]:
world_boundaries = g.content.get("2b93b06dc0dc4e809d3c8db5cb96ba69")
boundary_lyr = world_boundaries.layers[0]

In [49]:
phase2_list = ['Bethlehem, West Bank',
'Hebron, West Bank',
'Lima, Peru', #– metropolitan area
'Vinh Long, Vietnam',
'Khanh Hoa, Vietnam', #– focused on the districts of Nha Trang, Cam Ranh, Ninh Hoa and Van Ninh
'Semarang, Indonesia',
'Osh, Kyrgyz',
'Sfax, Tunisia',
'Sousse, Tunisia',
'Tunis, Tunisia',
'Batroun, Lebanon',
'Bint Jbeil, Lebanon',
'Montevideo, Uruguay',
'Nord, Haiti'#'Cap-Haitien, Haiti',
'Porto Alegre, Brazil', #– focused on the 4th district
'Ibadan, Nigeria',
'Mogadishu, Somalia',
'Garowe, Somalia ',
'Monrovia, Liberia',
"Abidjan, Cote d’Ivoire",
'Blantyre, Malawi',
'Rize, Turkey','Kahramanmaras, Turkey']

In [18]:
countries = SpatialDataFrame.from_layer(boundary_lyr)
countries.head()

,AFF_ISO,COUNTRYAFF,Country,FID,ISO,SHAPE
0,FK,United Kingdom,Falkland Islands,1,FK,"{'rings': [[[-6606596.26482308, -6862393.47442..."
1,FR,France,French Guiana,2,GF,"{'rings': [[[-6078465.19716979, 259355.8059185..."
2,GY,Guyana,Guyana,3,GY,"{'rings': [[[-6475746.20510377, 760125.6448030..."
3,CY,Cyprus,Cyprus,4,CY,"{'rings': [[[3703854.51647088, 4124479.2033668..."
4,PN,United Kingdom,Pitcairn,5,PN,"{'rings': [[[-14285877.1586672, -2793337.67101..."


In [47]:
search_result = g.content.search(query="title: city boundaries", item_type="Feature Layer", max_items = 50, outside_org=True)
search_result

[<Item title:"USA Major Cities" type:Feature Layer Collection owner:esri_dm>,
 <Item title:"Dallas County Cities" type:Feature Layer Collection owner:Leah.Bowyer_DC>,
 <Item title:"City Limits" type:Feature Layer Collection owner:JCGIS_Owner>,
 <Item title:"World Cities" type:Feature Layer Collection owner:esri_dm>,
 <Item title:"City Limits" type:Feature Layer Collection owner:slace>,
 <Item title:"Aggregation of Drug Incidents of Portland City in 2012 to Subset_boundary" type:Feature Layer Collection owner:geog4673_06>,
 <Item title:"Current Metropolitan New England City and Town Areas" type:Feature Layer Collection owner:cmenzel_dcdev>,
 <Item title:"City_of_Portland" type:Feature Layer Collection owner:arkemac_reed>,
 <Item title:"City Council Districts" type:Feature Layer Collection owner:cocgis_admin>,
 <Item title:"Grand Junction City_Limits" type:Feature Layer Collection owner:stapia_hla>,
 <Item title:"Boundaries _ City Info" type:Feature Layer Collection owner:savannah.winsta

In [27]:
res =search_result[1]

In [30]:
states = g.content.get(res.id)
states_lyr = states.layers[0]

In [31]:
states = SpatialDataFrame.from_layer(states_lyr)

In [32]:
states.head()

,CAND_COUNT,NA2desc,NAM,OBJECTID,Shape__Area,Shape__Length,SHAPE
0,915,Kiribati,Abatiku,11,0.000235,0.081996,"{'rings': [[[173.794937000242, 0.3756109996479..."
1,197,Papua New Guinea,Ablingi Island,19,0.000233,0.058690,"{'rings': [[[150.070526000175, -6.310417000044..."
2,293,Australia,Acheron Island,28,0.000053,0.033337,"{'rings': [[[146.629302999934, -18.96991700006..."
3,975,New Zealand,Adele Island,43,0.000094,0.038418,"{'rings': [[[173.055359000318, -40.98599999990..."
4,272,United States,Adugak Island,51,0.000103,0.060943,"{'rings': [[[-169.174804999951, 52.89848699997..."


In [61]:
geo = states[states.NAM == "Lima"].SHAPE.iloc[0]

In [74]:
geo1 = SpatialDataFrame.from_featureclass(r"C:\Users\wb542830\Documents\ArcGIS_Python\AOI\Lima_AOI.shp")

In [1]:
#arcpy.Clip_management(pop_lyr, "-77.26902770946117 -12.520417214083125 -76.62148284915412 -11.572877884306877", 
#                      r"Documents\ArcGIS_Python\Lima_clip.tif", r"Documents\ArcGIS_Python\AOI\Lima_AOI.shp",
#                      "0", "ClippingGeometry", 
#                      "MAINTAIN_EXTENT")



In [130]:
folder = r"Documents\ArcGIS_Python\rasters"

## Extracting spatial reference so that clipped rasters have the same reference

sr = pop_lyr.properties.spatialReference

names_dict = {pop_lyr:"pop",elev_lyr:"elev"}

In [139]:
from arcgis.raster.functions import clip

def output_raster(city,layer):
    
    '''
    Function to output rasters for elevation or population, clipped to national borders
    '''
    path = r"Documents\ArcGIS_Python\AOI\{}_AOI.shp".format(city)
    
    geo = SpatialDataFrame.from_featureclass(path)
    
    filename = names_dict[layer]+'_'+str(city)+'.tif'

    AOI = geocode(city,out_sr=sr)[0]['extent']
    AOI['spatialReference'] = sr
    layer.extent = AOI

    clipped = clip(layer,geo.SHAPE.iloc[0])
    clipped.export_image(f='image', bbox = AOI, export_format='tiff',
                    save_folder = folder, save_file=filename)

In [140]:
output_raster('Lima', pop_lyr)

In [134]:
path = r"Documents\ArcGIS_Python\AOI\{}_AOI.shp".format('Lima')
    
geo = SpatialDataFrame.from_featureclass(path)

In [5]:
lima = Raster(r"Documents\ArcGIS_Python\rasters\pop_Lima.tif")

In [8]:
lima

C:\Users\wb542830\Documents\ArcGIS_Python\rasters\pop_Lima.tif